In [ ]:
import urllib
import gromacs as gmx
import nglview as nv
import mdtraj as md

In [ ]:
# Download the PDB file and remove water molecules
# pdb='1L2Y' 
pdb='1FHA' 
urllib.request.urlretrieve(f'https://files.rcsb.org/download/{pdb}.pdb',f'{pdb}.pdb')
! grep -v HOH {pdb}.pdb > {pdb}_clean.pdb
pdb = f'{pdb}_clean'

# Generating topology

In [ ]:
# Generate GROMACS-formatted structure file that contains all the atoms defined within the force field (coordinate file .gro) and topology file (.top)
# Chosed force field: AMBER
# Chosed water model: TIP3P
gmx.pdb2gmx(f=pdb+'.pdb',o=pdb+'.gro',water='tip3p',ff='amber99',p=pdb+'.top',ignh=True)

# Defining box and Solvating

In [ ]:
# Create a cubic box around the protein structure
# Chosen box type: dodecahedron
# Chosen distance between protein and the edge of the box (at least): 1.5
gmx.editconf(f=pdb+'.gro',o=pdb+'-box.gro',c=True,d='1.5', bt='dodecahedron')

In [ ]:
# Solvate the system by adding water molecules (fill the box with water)
gmx.solvate(cp=pdb+'-box.gro',cs='spc216.gro',o=pdb+'-solv.gro',p=pdb+'.top')

# Adding ions

In [ ]:
# Generate a parameter file (ions.mdp) for adding ions to neutralize the system
# Changed coulombtype=cutoff to coulombtype=PME (as recommended in AMBER manuals)
with open('ions.mdp','w') as ions:
    ions.write("""\
; ions.mdp - used as input into grompp to generate ions.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme	= Verlet    ; Buffered neighbor searching 
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
""")

# Prepare the ion addition simulation, create .tpr file for genion
gmx.grompp(f='ions.mdp',c=pdb+'-solv.gro',p=pdb+'.top',o='ions.tpr')

In [ ]:
# Create an index file for selecting solvent molecules
gmx.select(s=pdb+'-solv.gro',on='solv.ndx',select='SOL')

In [ ]:
# Add ions to neutralize the system
gmx.genion(s='ions.tpr',n='solv.ndx',o=pdb+'-ions.gro',p=pdb+'.top',pname='NA',nname='CL',neutral=True)

# Energy minimization

In [ ]:
# Generate a parameter file (minim.mdp) for energy minimization
with open('minim.mdp','w') as m:
    m.write("""\
; minim.mdp - used as input into grompp to generate em.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = steep         ; Algorithm (steep = steepest descent minimization)
emtol       = 1000.0        ; Stop minimization when the maximum force < 1000.0 kJ/mol/nm
emstep      = 0.01          ; Minimization step size
nsteps      = 50000         ; Maximum number of (minimization) steps to perform

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 1         ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

nstxout                 = 50         
nstvout                 = 0        
nstfout                 = 0
nstenergy               = 50         
""")

# Prepare the energy minimization simulation, create .tpr file for mdrun
gmx.grompp(f='minim.mdp',c=pdb+'-ions.gro',p=pdb+'.top',o='em.tpr')

In [ ]:
mdrun=gmx.MDrunnerK8s()

In [ ]:
ompthreads=2
mpiranks=1
# Execute the minimization using GROMACS MD engine
mdrun.run(pre={'cores':ompthreads*mpiranks,'gpus':1}, mdrunargs={'deffnm':'em','ntomp':ompthreads},ncores=mpiranks)
# generated files - em.log: ASCII-text log file of the EM process; em.edr: Binary energy file; em.trr: Binary full-precision trajectory; em.gro: Energy-minimized structure

In [ ]:
# Analyze energy terms
energy = gmx.energy(f='em.edr', o='potential.xvg', terms=[10])

In [ ]:
# plot the graph

# Equilibration

In [ ]:
# Generate a parameter file (nvt.mdp) for NVT equilibration
with open('nvt.mdp','w') as nvt:
    nvt.write("""\
; nvt.mdp - used as input into grompp to generate nvt.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = md            ; Algorithm (md = molecular dynamics)
nsteps      = 50000         ; Maximum number of steps to perform
dt          = 0.002         ; Time step (in ps)

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist         = 10        ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions

; Temperature coupling
tcoupl          = V-rescale ; Temperature coupling type
tc-grps         = Protein   ; Group(s) to couple separately
tau_t           = 0.1       ; Time constant (in ps) for coupling
ref_t           = 300       ; Reference temperature (in K)

; Velocity generation
gen_vel         = yes       ; Generate velocities at start
gen_temp        = 300       ; Initial temperature (in K)
gen_seed        = -1        ; Random seed for velocity generation

; Output control
nstxout         = 1000      ; Save coordinates every 1000 steps
nstvout         = 1000      ; Save velocities every 1000 steps
nstenergy       = 1000      ; Save energies every 1000 steps
""")
    
# Prepare the NVT equilibration simulation, create .tpr file for mdrun
gmx.grompp(f='nvt.mdp',c='em.gro',r='em.gro',p=pdb+'.top',o='nvt.tpr')

In [ ]:
# Execute the NVT equilibration using GROMACS MD engine
mdrun.run(pre={'cores':ompthreads*mpiranks,'gpus':1}, mdrunargs={'deffnm':'nvt','ntomp':ompthreads},ncores=mpiranks)

In [ ]:
# Analyze temperature progression
gmx.energy(f='nvt.edr',o='temperature.xvg',terms=[16])

In [ ]:
# Generate a parameter file (npt.mdp) for NPT equilibration
with open('npt.mdp', 'w') as npt:
    npt.write("""\
; Parameters describing what to do, when to stop and what to save
integrator  = md            ; Molecular dynamics (MD) integration algorithm
dt          = 0.002         ; Time step (picoseconds)
nsteps      = 50000         ; Maximum number of time steps to perform
nstxout     = 1000          ; Save coordinates every 1000 steps
nstvout     = 1000          ; Save velocities every 1000 steps
nstenergy   = 1000          ; Save energies every 1000 steps
nstlog      = 1000          ; Update log file every 1000 steps

; Parameters describing how to find the neighbors of each atom and how to calculate the interactions
nstlist     = 10            ; Frequency to update the neighbor list and long range forces
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type     = grid          ; Method to determine neighbor list (simple, grid)
coulombtype = PME           ; Treatment of long range electrostatic interactions
rcoulomb    = 1.0           ; Short-range electrostatic cut-off (nanometers)
rvdw        = 1.0           ; Short-range Van der Waals cut-off (nanometers)
pbc         = xyz           ; Periodic Boundary Conditions in all 3 dimensions

; Parameters related to temperature coupling
tcoupl      = V-rescale     ; Temperature coupling method
tc-grps     = System        ; Group(s) for temperature coupling
tau_t       = 0.1           ; Coupling time constant (picoseconds)
ref_t       = 300           ; Reference temperature (Kelvin)

; Parameters related to pressure coupling
pcoupl      = Parrinello-Rahman    ; Pressure coupling method
pcoupltype  = isotropic             ; Apply pressure isotropically
tau_p       = 2.0                   ; Pressure coupling time constant (picoseconds)
ref_p       = 1.0                   ; Reference pressure (bar)
compressibility = 4.5e-5            ; Isothermal compressibility (1/bar)

; Other options
gen_vel     = no            ; Do not generate velocities
continuation = yes          ; Continue from previous NVT equilibration
""")

# Prepare the NPT equilibration simulation, create .tpr file for mdrun
gmx.grompp(f='npt.mdp', c='nvt.gro', r='nvt.gro', t='nvt.cpt', p=pdb+'.top', o='npt.tpr')

In [ ]:
# Execute the NPT equilibration using GROMACS MD engine
mdrun.run(pre={'cores':ompthreads*mpiranks,'gpus':1}, mdrunargs={'deffnm':'npt','ntomp':ompthreads},ncores=mpiranks)

In [ ]:
# Analyze pressure progression
gmx.energy(f='npt.edr', o='pressure.xvg', properties=['Pressure'])

# Analyze density
gmx.energy(f='npt.edr', o='density.xvg', properties=['Density'])

# Production MD

In [ ]:
# Generate a parameter file (md.mdp) for MD simulation
with open('md.mdp', 'w') as md:
    md.write("""\
; md.mdp - used as input into grompp to generate md_0_1.tpr
; Parameters describing what to do, when to stop and what to save
integrator  = md            ; Molecular dynamics integration method (md: molecular dynamics)
dt          = 0.002         ; Time step in ps
nsteps      = 500000        ; Total number of time steps (1 ns)
nstxout     = 5000          ; Frequency to write coordinates to trajectory file (1 snapshot every 10 ps)
nstvout     = 5000          ; Frequency to write velocities to trajectory file
nstenergy   = 5000          ; Frequency to write energies to energy file
nstlog      = 5000          ; Frequency to write data to log file
nstxtcout   = 5000          ; Frequency to write coordinates to xtc trajectory file (for visualization)
continuation= yes           ; Continue from a previous run (required when continuing from NPT)
constraint_algorithm = lincs   ; Constraint algorithm for bond lengths and angles
constraints = all-bonds       ; All bonds (including hydrogen bonds) constrained
lincs_iter  = 1               ; Number of iterations for the LINCS algorithm
lincs_order = 4               ; Highest order in the expansion of the constraint coupling matrix
cutoff-scheme   = Verlet    ; Buffered neighbor searching
ns_type         = grid      ; Method to determine neighbor list (simple, grid)
coulombtype     = PME       ; Treatment of long range electrostatic interactions
rcoulomb        = 1.0       ; Short-range electrostatic cut-off
rvdw            = 1.0       ; Short-range Van der Waals cut-off
pbc             = xyz       ; Periodic Boundary Conditions in all 3 dimensions
tcoupl          = V-rescale ; Velocity rescaling thermostat
tc-grps         = Protein   ; Apply temperature coupling to protein group
tau_t           = 0.1       ; Time constant for temperature coupling (in ps)
ref_t           = 300       ; Reference temperature for thermostat (in K)
pcoupl          = Parrinello-Rahman ; Parrinello-Rahman pressure coupling
pcoupltype      = isotropic  ; Apply isotropic pressure coupling
tau_p           = 2.0       ; Time constant for pressure coupling (in ps)
ref_p           = 1.0       ; Reference pressure for pressure coupling (in bar)
compressibility = 4.5e-5    ; Isothermal compressibility of water (in bar^-1)
gen_vel         = no        ; Do not generate velocities at the beginning (read from NPT)
""")

# Prepare the MD simulation, create .tpr file for mdrun
gmx.grompp(f='md.mdp', c='npt.gro', t='npt.cpt', p=pdb+'.top', o='md_0_1.tpr')

In [ ]:
# Execute the MD simulation
mdrun.run(pre={'cores':ompthreads*mpiranks,'gpus':1}, mdrunargs={'deffnm':'md_0_1','ntomp':ompthreads},ncores=mpiranks)

# Analysis

In [ ]:
# Create an index file (protein.ndx) containing only atoms that belong to the protein
# gmx.select(s=pdb+'-ions.gro',on='protein.ndx',select='Protein')
gmx.select(s='md_0_1.gro',on='protein.ndx',select='Protein')

In [ ]:
# Convert the trajectory file to XTC format
# gmx.trjconv(s=pdb+'-ions.gro',f='em.trr',n='protein.ndx',o='em-protein.xtc')

In [ ]:
# Convert the trajectory file to XTC format
# gmx.trjconv(s=pdb+'-ions.gro',f='md_0_1.trr',n='protein.ndx',o='em-protein.xtc')
gmx.trjconv(s='md_0_1.gro',f='md_0_1.trr',n='protein.ndx',o='md_0_1.xtc')

In [ ]:
# Process trajectory to remove the effects of periodic boundary conditions
gmx.trjconv(s='md_0_1.tpr', f='md_0_1.xtc', o='md_0_1_noPBC.xtc', pbc='mol', center=True, group=1, ogroup=0)

In [ ]:
# Calculate RMSD
gmx.rms(s='md_0_1.tpr', f='md_0_1_noPBC.xtc', o='rmsd.xvg', tu='ns', group=4, fitgroup=4)

In [ ]:
# Calculate RMSD relative to the crystal structure
gmx.rms(s='em.tpr', f='md_0_1_noPBC.xtc', o='rmsd_xtal.xvg', tu='ns', group=4, fitgroup=4)

In [ ]:
# Run gyrate analysis
gmx.gyrate(s='md_0_1.tpr', f='md_0_1_noPBC.xtc', o='gyrate.xvg')

# Visualization

In [ ]:
# Load the trajectory file and visualize it using NGLView
# tr=md.load('em-protein.xtc',top=pdb+'.gro')
tr=md.load('md_0_1.xtc',top=pdb+'.top')

nv.show_mdtraj(tr)